In [1]:
%load_ext jupyter_black
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import ipyleaflet
from ipyleaflet import Marker, MarkerCluster, Polygon
import alphashape
from descartes import PolygonPatch
import os
import sys
import pandas as pd
import numpy as np
from descartes import PolygonPatch
import matplotlib.pyplot as plt
import io
import geemap
import requests
import datetime
import csv

sys.path.insert(0, os.path.dirname(os.getcwd()))
import alphashape

In [2]:
from google.auth import compute_engine
import ee

credentials = compute_engine.Credentials(
    scopes=["https://www.googleapis.com/auth/earthengine"]
)
ee.Initialize(credentials)

In [3]:
def maskS2clouds(image):
    qa = image.select("QA60")

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000)

In [4]:
Sentinel_rgb_viz = {
    "bands": ["B4", "B3", "B2"],
    "min": 0.0,
    "max": 0.3,
}

sentinel2 = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterDate("2022-01-01", "2022-01-31")
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
    .map(maskS2clouds)
)

In [5]:
def get_point_parquet(bucket_name, path, filename):
    from google.cloud import storage
    import pandas as pd

    # bucket_name = "wind-turbine-project-thibaud"

    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    # When you have your files in a subfolder of the bucket.
    my_prefix = path  # the name of the subfolder
    blobs = bucket.list_blobs(prefix=my_prefix, delimiter="/")
    for blob in blobs:
        if (
            blob.name != my_prefix
        ) and blob.name == path + filename:  # ignoring the subfolder itself
            file_name = blob.name.replace(my_prefix, "")
            blob.download_to_filename(file_name)  # download the file to the machine
            dff = pd.read_parquet(file_name)  # load the data
    return dff

In [6]:
def draw_shape(cloud_of_points, alpha):
    alpha_shape = alphashape.alphashape(cloud_of_points, alpha)
    fig, ax = plt.subplots()
    ax.scatter(*zip(*cloud_of_points))
    ax.add_patch(PolygonPatch(alpha_shape, alpha=0.2))
    try:
        ax.add_patch(PolygonPatch(alpha_shape, alpha=alpha))
    except:
        print("exemption")
    return alpha_shape

In [7]:
def wind_farm_outline(cloud_of_points, alpha):
    alpha_shape = alphashape.alphashape(cloud_of_points, alpha)
    alpha_coord = np.asarray(alpha_shape.exterior.coords)
    alpha_coord = [np.asarray(list(zip(alpha_coord[:, 1], alpha_coord[:, 0]))).tolist()]
    return alpha_coord

In [8]:
def update_map(index):
    row = active_offshore_db_accurate.iloc[index]
    lon_value.value = row["Longitude"]
    lat_value.value = row["Latitude"]
    map_zone_ee_map(lon, lat)

In [9]:
def map_zone_ee_map(lon, lat, buffer, StartDate, EndDate):
    # Create the map
    global map
    map = geemap.Map()
    map.remove_layer(map.layers[0])
    map.add_basemap("OpenStreetMap")
    map.setCenter(lon, lat, 11.5)
    Sentinel_rgb_viz = {
        "bands": ["B4", "B3", "B2"],
        "min": 0.0,
        "max": 0.3,
    }
    sentinel2 = (
        ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
        .filterDate(StartDate, EndDate)
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
        .map(maskS2clouds)
    )
    # Region of interest
    roi = ee.Geometry.Rectangle(
        [lon - buffer, lat - buffer, lon + buffer, lat + buffer]
    )
    map.addLayer(roi, name="region of interest", opacity=0.2)
    # Get the RGB image of the ROI
    image = sentinel2.median().clip(roi).select(Sentinel_rgb_viz["bands"])
    map.addLayer(image, Sentinel_rgb_viz, name="Sentinel RGB", opacity=0.5)
    # Get the wind turbine in the ROI
    offshore_turbine = get_point_parquet(
        "wind-turbine-project-thibaud",
        "Open street Map data/",
        "offshore_wind_turbine_g2.parquet",
    )
    wt_in_zone = offshore_turbine[
        (
            (offshore_turbine["longitude"] < lon + buffer)
            & (offshore_turbine["longitude"] > lon - buffer)
        )
        & (
            (offshore_turbine["latitude"] < lat + buffer)
            & (offshore_turbine["latitude"] > lat - buffer)
        )
    ]
    marker_cluster = MarkerCluster(
        markers=[
            Marker(
                location=(wt_in_zone["latitude"][index], wt_in_zone["longitude"][index])
            )
            for index, row in wt_in_zone.iterrows()
        ],
        name="wind turbine",
    )
    # map.add(marker_cluster)
    # create a polygon from the OSM wind turbine data
    try:
        outline = wind_farm_outline(wt_in_zone[["latitude", "longitude"]], 0.2)
        global polygon
        polygon = ee.Geometry.Polygon(outline)
        map.addLayer(polygon, name="Offshore Wind Farm")
    except:
        pass
    # Allow to modify the polygon
    draw_control = ipyleaflet.DrawControl()

    def handle_draw(target, action, geo_json):
        # This is called when the user draws a new shape.
        # You can replace the polygon with the new shape here.
        polygon = geo_json["geometry"]

    # draw_control.on_draw(handle_draw)
    # map.add_control(draw_control)

    def update_polygon(b):
        polygon_collection = []
        for feature in map.draw_features:
            polygon_collection.append(
                ee.Geometry.Polygon(feature.getInfo()["geometry"]["coordinates"])
            )
        polys = ee.Geometry.MultiPolygon(polygon_collection)
        map.addLayer(polys, name="Offshore Wind Farms", opacity=0.5)
        # global polygon
        # outline = polygon["coordinates"]
        # polygon = ee.Geometry.Polygon(outline)
        # print(outline)
        # map.addLayer(polygon, name="Offshore Wind Farm")

    update_button = widgets.Button(description="Update Polygon")
    update_button.on_click(update_polygon)
    display(update_button)

    # Add the Save button
    save_button = widgets.Button(description="Save")

    def saving_info(b):
        polygons = []
        if map.draw_features == []:
            polygons.append(polygon.getInfo())
        else:
            for feature in map.draw_features:
                polygons.append(feature.geometry().getInfo())
        dic = {
            "Field ID": current_index,
            "Longitude": lon_value.value,
            "Latitude": lat_value.value,
            "Buffer": buffer_value.value,
            "Start date": StartDate,
            "End date": EndDate,
            "ROI": roi.toGeoJSONString(),
            "Wind farm polygon": polygons,
        }

        with open(
            os.path.realpath(
                os.path.join(
                    os.path.curdir,
                    "..",
                    "data",
                    "processed",
                    "Wind_farm_lon_lat_geometry.csv",
                )
            ),
            "a",
        ) as f_object:
            dictwriter_object = csv.DictWriter(f_object, fieldnames=dic.keys())
            dictwriter_object.writerow(dic)
            f_object.close()

    save_button.on_click(saving_info)
    display(save_button)

    # Next wind farm
    def handle_next(b):
        global current_index
        current_index += 1
        if current_index >= len(active_offshore_db_accurate):
            print("No more data")
        else:
            update_map(current_index)

    next_button = widgets.Button(description="Next")
    next_button.on_click(handle_next)
    display(next_button)
    return map

In [10]:
raw_db = pd.read_excel(
    "Global-Wind-Power-Tracker-May-2023.xlsx", "Data", usecols="B,C,G,H,I,J,P,Q,R"
)
turbine_type = ["offshore floating", "offshore hard mount", "offshore mount unknown"]
offshore_db = raw_db[raw_db["Installation Type"].isin(turbine_type)]
active_offshore_db = offshore_db[offshore_db["Status"] == "operating"]
active_offshore_db_accurate = active_offshore_db[
    active_offshore_db["Location accuracy"] == "exact"
]

In [24]:
active_offshore_db_accurate[
    active_offshore_db_accurate["Country"] == "United Kingdom"
].iloc[:40]

,Country,Project Name,Capacity (MW),Installation Type,Status,Start year,Latitude,Longitude,Location accuracy
19641,United Kingdom,Aberdeen offshore wind farm,97.0,offshore mount unknown,operating,2018.0,57.2225,-1.9728,exact
19734,United Kingdom,Barrow wind farm,90.0,offshore hard mount,operating,2006.0,53.9915,-3.2983,exact
19737,United Kingdom,Beatrice wind farm,588.0,offshore hard mount,operating,2019.0,58.0882,-2.9502,exact
19795,United Kingdom,Blyth Offshore wind farm,42.0,offshore hard mount,operating,2017.0,55.1670,-1.3644,exact
19841,United Kingdom,Burbo Bank wind farm,90.0,offshore hard mount,operating,2007.0,53.4882,-3.1849,exact
19842,United Kingdom,Burbo Bank wind farm,258.0,offshore hard mount,operating,2017.0,53.4801,-3.2701,exact
20062,United Kingdom,Dudgeon wind farm,402.0,offshore hard mount,operating,2017.0,53.1175,0.6135,exact
20093,United Kingdom,East Anglia wind farm,714.0,offshore hard mount,operating,2020.0,52.1370,2.1728,exact
20114,United Kingdom,European Offshore Wind Deployment Centre (Eowd...,97.0,offshore mount unknown,operating,2018.0,57.1462,-2.0728,exact
20166,United Kingdom,Galloper wind farm,353.0,offshore hard mount,operating,2018.0,51.8801,2.0401,exact


In [34]:
active_offshore_db_accurate.iloc[163:165]

,Country,Project Name,Capacity (MW),Installation Type,Status,Start year,Latitude,Longitude,Location accuracy
20359,United Kingdom,Kentish Flats wind farm,90.0,offshore hard mount,operating,2005.0,51.4606,1.0938,exact
20360,United Kingdom,Kentish Flats wind farm,50.0,offshore hard mount,operating,2015.0,51.4501,1.0801,exact


In [13]:
os.path.abspath(os.curdir)

'/home/jupyter/ee_segmentation/notebooks/data prep'

In [14]:
def on_modify_clicked(b):
    if map.draw_last_feature is not None:
        polygon = map2.draw_last_feature
        # map.addLayer(polygon, name="Kentish Flat Offshore Wind Farm")
        return polygon

In [15]:
def on_save_clicked(b):
    # print(f"Longitude: {lon_value.value}")
    # print(f"Latitude2: {lat_value.value}")
    # print(f"Buffer: {buffer_value.value}")
    # print(f"Start date: {Start_date_value.value}")
    # print(f"End date: {End_date_value.value}")
    # print(f"ROI: {square}")
    # print(f"Wind farm polygon: {polygon}")
    print("!")
    dic = {
        "Longitude": lon_value.value,
        "Latitude": lat_value.value,
        "Buffer": buffer_value.value,
        "Start date": Start_date_value.value,
        "End date": End_date_value.value,
        "ROI": roi,
        "Wind farm polygon": polygon,
    }
    print(dic)
    with open("Data_info.csv", "a") as f_object:
        dictwriter_object = csv.DictWriter(f_object, fieldnames=dic.keys())
        dictwriter_object.writerow(dic)
        f_object.close()

In [16]:
def on_next_click(b):
    index = +1

In [29]:
global index
current_index = 154
e = 0.05

In [30]:
lon_value = widgets.BoundedFloatText(
    value=active_offshore_db_accurate.iloc[current_index]["Longitude"],
    min=-180,
    max=180,
    step=0.001,
    description="Longitude:",
    disabled=False,
)
lat_value = widgets.BoundedFloatText(
    value=active_offshore_db_accurate.iloc[current_index]["Latitude"],
    min=-90,
    max=90,
    step=0.001,
    description="Latitude:",
    disabled=False,
)
buffer_value = widgets.BoundedFloatText(
    value=e, min=0, max=5, step=0.001, description="Buffer:", disabled=False
)
Start_date_value = widgets.DatePicker(
    value=datetime.date(2022, 1, 1), description="Start Date", disabled=False
)
End_date_value = widgets.DatePicker(
    value=datetime.date(2022, 1, 31), description="End Date", disabled=False
)

# Next = widgets.ToggleButtons(
#     options=["Save", "Next"],
#     disabled=False,
#     button_style="info",  # 'success', 'info', 'warning', 'danger' or ''
#     tooltips=["Description of slow", "Description of regular", "Description of fast"],
#     #     icons=['check'] * 3
# )
Modify_Shape = widgets.Button(description="Modify shape")
Save = widgets.Button(description="Save")
Next = widgets.Button(description="Next")

In [31]:
Save.on_click(on_save_clicked)
Modify_Shape.on_click(on_modify_clicked)
Next.on_click(on_next_click)

In [32]:
# print(
#     active_offshore_db_accurate.iloc[index]["Latitude"],
#     active_offshore_db_accurate.iloc[index]["Longitude"],
# )

In [33]:
display(
    widgets.interact(
        map_zone_ee_map,
        lon=lon_value,
        lat=lat_value,
        buffer=buffer_value,
        StartDate="2022-10-01",
        EndDate="2023-01-30",
        # polygon = Modify_Shape,
    ),
)

interactive(children=(BoundedFloatText(value=1.218, description='Longitude:', max=180.0, min=-180.0, step=0.00…

<function __main__.map_zone_ee_map(lon, lat, buffer, StartDate, EndDate)>

In [22]:
current_index

25